In [1]:
!python3.6 -m pip install tensorboardX
!python3.6 -m pip install torch
!python3.6 -m pip install librosa
!apt-get install libsndfile1 --yes

     |████████████████████████████████| 204kB 1.0MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 734.6MB 125kB/s  eta 0:00:01     |███████████▋                    | 266.5MB 12.2MB/s eta 0:00:39     |███████████████▉                | 364.0MB 10.5MB/s eta 0:00:36     |████████████████████████████▍   | 652.5MB 12.1MB/s eta 0:00:07
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 1.6MB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 26.1MB 12.2MB/s eta 0:00:01
     |████████████████████████████████| 7.1MB 11.2MB/s eta 0:00:01
     |████████████████████████████████| 296kB 5.6MB/s eta 0:00:01
     |████████████████████████████████| 327kB 14.1MB/s eta 0:00:01
     |████████████████████████████████| 3.7MB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 20.2MB 10.6MB/s eta 0:00:01
     |████████████████████████████████| 399

Selecting previously unselected package libogg0:amd64.
(Reading database ... 15402 files and directories currently installed.)
Preparing to unpack .../libogg0_1.3.2-1_amd64.deb ...
Unpacking libogg0:amd64 (1.3.2-1) ...
Selecting previously unselected package libflac8:amd64.
Preparing to unpack .../libflac8_1.3.2-1_amd64.deb ...
Unpacking libflac8:amd64 (1.3.2-1) ...
Selecting previously unselected package libvorbis0a:amd64.
Preparing to unpack .../libvorbis0a_1.3.5-4.2_amd64.deb ...
Unpacking libvorbis0a:amd64 (1.3.5-4.2) ...
Selecting previously unselected package libvorbisenc2:amd64.
Preparing to unpack .../libvorbisenc2_1.3.5-4.2_amd64.deb ...
Unpacking libvorbisenc2:amd64 (1.3.5-4.2) ...
Selecting previously unselected package libsndfile1:amd64.
Preparing to unpack .../libsndfile1_1.0.28-4ubuntu0.18.04.1_amd64.deb ...
Unpacking libsndfile1:amd64 (1.0.28-4ubuntu0.18.04.1) ...
Setting up libogg0:amd64 (1.3.2-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libvo

# Test code copy

In [2]:
import torch
from torch import optim
from torch.autograd import Variable
import numpy as np
import pickle
from utils import Hps
from preprocess.tacotron.norm_utils import spectrogram2wav, get_spectrogram
from scipy.io.wavfile import write
import glob
import os
from solver import Solver
import librosa
import librosa.display
from plots import plot
%matplotlib inline

args_hps='vctk.json'
args_model="model_vctk/fixed_8bit_16k_paused_reusmed_latschediters_50k/model_vctk-80000"
args_source="test/p225_001.wav"
args_source_8="test/p225_001-8.wav"
args_target=0
args_output="test/output.wav"
args_sample_rate=16000
args_use_gen=False
wavenet_mel = True

hps = Hps()
hps.load(args_hps)
hps_tuple = hps.get_tuple()
solver = Solver(hps_tuple, None)
solver.load_model(args_model)
spec = get_spectrogram(args_source, wavenet_mel)
spec_8 = get_spectrogram(args_source_8, wavenet_mel)

spec_expand = np.expand_dims(spec_8, axis=0)
spec_tensor = torch.from_numpy(spec_expand).type(torch.FloatTensor)
c = Variable(torch.from_numpy(np.array([int(args_target)]))).cuda()
result = solver.test_step(spec_tensor, c, gen=args_use_gen)
result = result.squeeze(axis=0).transpose((1, 0))
wav_data = spectrogram2wav(result)
write(args_output, rate=args_sample_rate, data=wav_data)

plot(spec.T, spec_8.T, result.T)

load model from model_vctk/fixed_8bit_16k_paused_reusmed_latschediters_50k/model_vctk-80000


RuntimeError: Given groups=1, weight of size 128 513 1, expected input[1, 80, 139] to have 513 channels, but got 80 channels instead

# Train code copy

In [10]:
import torch
from torch import optim
from torch.autograd import Variable
import numpy as np
import pickle
from utils import Hps
from utils import DataLoader
from utils import Logger
from utils import SingleDataset
from solver import Solver
import argparse
%matplotlib inline

args_load_model=False
args_flag='train'
args_hps_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk.json"
args_dataset_path_trg="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/data.h5"
args_dataset_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/data-8-wavenet.h5"
args_index_path_trg="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/index.json"
args_index_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/index.json"
args_output_model_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/model_vctk/model_vctk"
args_load_model_path="model_vctk/model_vctk-32000"

hps = Hps()
hps.load(args_hps_path)
hps_tuple = hps.get_tuple()
dataset = SingleDataset(args_dataset_path, args_index_path, seg_len=hps_tuple.seg_len)
dataset_trg = SingleDataset(args_dataset_path_trg, args_index_path_trg, seg_len=hps_tuple.seg_len)
data_loader = DataLoader(dataset, dataset_trg)

solver = Solver(hps_tuple, data_loader)
if args_load_model:
    solver.load_model(args_load_model_path)

solver.train(args_output_model_path, args_flag, mode='pretrain_G')
solver.train(args_output_model_path, args_flag, mode='pretrain_D')
solver.train(args_output_model_path, args_flag, mode='train')
solver.train(args_output_model_path, args_flag, mode='patchGAN')

pre_G:[000001/008000], loss_rec=0.479
pre_G:[000002/008000], loss_rec=0.464
pre_G:[000003/008000], loss_rec=0.420
pre_G:[000004/008000], loss_rec=0.433
pre_G:[000005/008000], loss_rec=0.401
pre_G:[000006/008000], loss_rec=0.392
pre_G:[000007/008000], loss_rec=0.365
pre_G:[000008/008000], loss_rec=0.378
pre_G:[000009/008000], loss_rec=0.344
pre_G:[000010/008000], loss_rec=0.337
pre_G:[000011/008000], loss_rec=0.322
pre_G:[000012/008000], loss_rec=0.320
pre_G:[000013/008000], loss_rec=0.305
pre_G:[000014/008000], loss_rec=0.296
pre_G:[000015/008000], loss_rec=0.297
pre_G:[000016/008000], loss_rec=0.278
pre_G:[000017/008000], loss_rec=0.258
pre_G:[000018/008000], loss_rec=0.266
pre_G:[000019/008000], loss_rec=0.260
pre_G:[000020/008000], loss_rec=0.245
pre_G:[000021/008000], loss_rec=0.240
pre_G:[000022/008000], loss_rec=0.237
pre_G:[000023/008000], loss_rec=0.232
pre_G:[000024/008000], loss_rec=0.231
pre_G:[000025/008000], loss_rec=0.218
pre_G:[000026/008000], loss_rec=0.218
pre_G:[00002

pre_G:[000217/008000], loss_rec=0.098
pre_G:[000218/008000], loss_rec=0.099
pre_G:[000219/008000], loss_rec=0.100
pre_G:[000220/008000], loss_rec=0.104
pre_G:[000221/008000], loss_rec=0.106
pre_G:[000222/008000], loss_rec=0.113
pre_G:[000223/008000], loss_rec=0.101
pre_G:[000224/008000], loss_rec=0.100
pre_G:[000225/008000], loss_rec=0.102
pre_G:[000226/008000], loss_rec=0.100
pre_G:[000227/008000], loss_rec=0.101
pre_G:[000228/008000], loss_rec=0.100
pre_G:[000229/008000], loss_rec=0.106
pre_G:[000230/008000], loss_rec=0.101
pre_G:[000231/008000], loss_rec=0.101
pre_G:[000232/008000], loss_rec=0.101
pre_G:[000233/008000], loss_rec=0.098
pre_G:[000234/008000], loss_rec=0.101
pre_G:[000235/008000], loss_rec=0.102
pre_G:[000236/008000], loss_rec=0.101
pre_G:[000237/008000], loss_rec=0.098
pre_G:[000238/008000], loss_rec=0.099
pre_G:[000239/008000], loss_rec=0.108
pre_G:[000240/008000], loss_rec=0.101
pre_G:[000241/008000], loss_rec=0.102
pre_G:[000242/008000], loss_rec=0.094
pre_G:[00024

pre_G:[000433/008000], loss_rec=0.089
pre_G:[000434/008000], loss_rec=0.092
pre_G:[000435/008000], loss_rec=0.088
pre_G:[000436/008000], loss_rec=0.089
pre_G:[000437/008000], loss_rec=0.090
pre_G:[000438/008000], loss_rec=0.086
pre_G:[000439/008000], loss_rec=0.089
pre_G:[000440/008000], loss_rec=0.091
pre_G:[000441/008000], loss_rec=0.093
pre_G:[000442/008000], loss_rec=0.090
pre_G:[000443/008000], loss_rec=0.086
pre_G:[000444/008000], loss_rec=0.087
pre_G:[000445/008000], loss_rec=0.084
pre_G:[000446/008000], loss_rec=0.084
pre_G:[000447/008000], loss_rec=0.085
pre_G:[000448/008000], loss_rec=0.085
pre_G:[000449/008000], loss_rec=0.089
pre_G:[000450/008000], loss_rec=0.088
pre_G:[000451/008000], loss_rec=0.091
pre_G:[000452/008000], loss_rec=0.093
pre_G:[000453/008000], loss_rec=0.088
pre_G:[000454/008000], loss_rec=0.091
pre_G:[000455/008000], loss_rec=0.091
pre_G:[000456/008000], loss_rec=0.087
pre_G:[000457/008000], loss_rec=0.087
pre_G:[000458/008000], loss_rec=0.087
pre_G:[00045

pre_G:[000649/008000], loss_rec=0.082
pre_G:[000650/008000], loss_rec=0.085
pre_G:[000651/008000], loss_rec=0.086
pre_G:[000652/008000], loss_rec=0.084
pre_G:[000653/008000], loss_rec=0.086
pre_G:[000654/008000], loss_rec=0.087
pre_G:[000655/008000], loss_rec=0.091
pre_G:[000656/008000], loss_rec=0.086
pre_G:[000657/008000], loss_rec=0.081
pre_G:[000658/008000], loss_rec=0.087
pre_G:[000659/008000], loss_rec=0.081
pre_G:[000660/008000], loss_rec=0.083
pre_G:[000661/008000], loss_rec=0.083
pre_G:[000662/008000], loss_rec=0.087
pre_G:[000663/008000], loss_rec=0.082
pre_G:[000664/008000], loss_rec=0.084
pre_G:[000665/008000], loss_rec=0.081
pre_G:[000666/008000], loss_rec=0.082
pre_G:[000667/008000], loss_rec=0.088
pre_G:[000668/008000], loss_rec=0.087
pre_G:[000669/008000], loss_rec=0.083
pre_G:[000670/008000], loss_rec=0.083
pre_G:[000671/008000], loss_rec=0.079
pre_G:[000672/008000], loss_rec=0.080
pre_G:[000673/008000], loss_rec=0.084
pre_G:[000674/008000], loss_rec=0.083
pre_G:[00067

pre_G:[000865/008000], loss_rec=0.080
pre_G:[000866/008000], loss_rec=0.080
pre_G:[000867/008000], loss_rec=0.084
pre_G:[000868/008000], loss_rec=0.080
pre_G:[000869/008000], loss_rec=0.073
pre_G:[000870/008000], loss_rec=0.079
pre_G:[000871/008000], loss_rec=0.078
pre_G:[000872/008000], loss_rec=0.085
pre_G:[000873/008000], loss_rec=0.082
pre_G:[000874/008000], loss_rec=0.078
pre_G:[000875/008000], loss_rec=0.081
pre_G:[000876/008000], loss_rec=0.081
pre_G:[000877/008000], loss_rec=0.077
pre_G:[000878/008000], loss_rec=0.079
pre_G:[000879/008000], loss_rec=0.083
pre_G:[000880/008000], loss_rec=0.080
pre_G:[000881/008000], loss_rec=0.084
pre_G:[000882/008000], loss_rec=0.081
pre_G:[000883/008000], loss_rec=0.080
pre_G:[000884/008000], loss_rec=0.083
pre_G:[000885/008000], loss_rec=0.084
pre_G:[000886/008000], loss_rec=0.081
pre_G:[000887/008000], loss_rec=0.084
pre_G:[000888/008000], loss_rec=0.082
pre_G:[000889/008000], loss_rec=0.080
pre_G:[000890/008000], loss_rec=0.079
pre_G:[00089

...

pre_G:[001945/008000], loss_rec=0.074
pre_G:[001946/008000], loss_rec=0.075
pre_G:[001947/008000], loss_rec=0.075
pre_G:[001948/008000], loss_rec=0.076
pre_G:[001949/008000], loss_rec=0.073
pre_G:[001950/008000], loss_rec=0.073
pre_G:[001951/008000], loss_rec=0.069
pre_G:[001952/008000], loss_rec=0.070
pre_G:[001953/008000], loss_rec=0.072
pre_G:[001954/008000], loss_rec=0.072
pre_G:[001955/008000], loss_rec=0.071
pre_G:[001956/008000], loss_rec=0.072
pre_G:[001957/008000], loss_rec=0.070
pre_G:[001958/008000], loss_rec=0.074
pre_G:[001959/008000], loss_rec=0.072
pre_G:[001960/008000], loss_rec=0.072
pre_G:[001961/008000], loss_rec=0.078
pre_G:[001962/008000], loss_rec=0.071
pre_G:[001963/008000], loss_rec=0.071
pre_G:[001964/008000], loss_rec=0.074
pre_G:[001965/008000], loss_rec=0.073
pre_G:[001966/008000], loss_rec=0.074
pre_G:[001967/008000], loss_rec=0.073
pre_G:[001968/008000], loss_rec=0.074
pre_G:[001969/008000], loss_rec=0.079
pre_G:[001970/008000], loss_rec=0.069
pre_G:[00197

pre_G:[002161/008000], loss_rec=0.070
pre_G:[002162/008000], loss_rec=0.071
pre_G:[002163/008000], loss_rec=0.071
pre_G:[002164/008000], loss_rec=0.076
pre_G:[002165/008000], loss_rec=0.076
pre_G:[002166/008000], loss_rec=0.076
pre_G:[002167/008000], loss_rec=0.072
pre_G:[002168/008000], loss_rec=0.074
pre_G:[002169/008000], loss_rec=0.070
pre_G:[002170/008000], loss_rec=0.077
pre_G:[002171/008000], loss_rec=0.074
pre_G:[002172/008000], loss_rec=0.073
pre_G:[002173/008000], loss_rec=0.074
pre_G:[002174/008000], loss_rec=0.070
pre_G:[002175/008000], loss_rec=0.071
pre_G:[002176/008000], loss_rec=0.074
pre_G:[002177/008000], loss_rec=0.073
pre_G:[002178/008000], loss_rec=0.072
pre_G:[002179/008000], loss_rec=0.069
pre_G:[002180/008000], loss_rec=0.071
pre_G:[002181/008000], loss_rec=0.074
pre_G:[002182/008000], loss_rec=0.074
pre_G:[002183/008000], loss_rec=0.068
pre_G:[002184/008000], loss_rec=0.075
pre_G:[002185/008000], loss_rec=0.071
pre_G:[002186/008000], loss_rec=0.070
pre_G:[00218

pre_G:[002377/008000], loss_rec=0.075
pre_G:[002378/008000], loss_rec=0.069
pre_G:[002379/008000], loss_rec=0.072
pre_G:[002380/008000], loss_rec=0.070
pre_G:[002381/008000], loss_rec=0.076
pre_G:[002382/008000], loss_rec=0.069
pre_G:[002383/008000], loss_rec=0.075
pre_G:[002384/008000], loss_rec=0.075
pre_G:[002385/008000], loss_rec=0.076
pre_G:[002386/008000], loss_rec=0.073
pre_G:[002387/008000], loss_rec=0.071
pre_G:[002388/008000], loss_rec=0.070
pre_G:[002389/008000], loss_rec=0.072
pre_G:[002390/008000], loss_rec=0.075
pre_G:[002391/008000], loss_rec=0.070
pre_G:[002392/008000], loss_rec=0.071
pre_G:[002393/008000], loss_rec=0.070
pre_G:[002394/008000], loss_rec=0.075
pre_G:[002395/008000], loss_rec=0.071
pre_G:[002396/008000], loss_rec=0.074
pre_G:[002397/008000], loss_rec=0.071
pre_G:[002398/008000], loss_rec=0.065
pre_G:[002399/008000], loss_rec=0.069
pre_G:[002400/008000], loss_rec=0.075
pre_G:[002401/008000], loss_rec=0.069
pre_G:[002402/008000], loss_rec=0.073
pre_G:[00240

pre_G:[002593/008000], loss_rec=0.068
pre_G:[002594/008000], loss_rec=0.072
pre_G:[002595/008000], loss_rec=0.072
pre_G:[002596/008000], loss_rec=0.068
pre_G:[002597/008000], loss_rec=0.072
pre_G:[002598/008000], loss_rec=0.074
pre_G:[002599/008000], loss_rec=0.072
pre_G:[002600/008000], loss_rec=0.074
pre_G:[002601/008000], loss_rec=0.069
pre_G:[002602/008000], loss_rec=0.072
pre_G:[002603/008000], loss_rec=0.071
pre_G:[002604/008000], loss_rec=0.071
pre_G:[002605/008000], loss_rec=0.075
pre_G:[002606/008000], loss_rec=0.070
pre_G:[002607/008000], loss_rec=0.074
pre_G:[002608/008000], loss_rec=0.074
pre_G:[002609/008000], loss_rec=0.072
pre_G:[002610/008000], loss_rec=0.071
pre_G:[002611/008000], loss_rec=0.070
pre_G:[002612/008000], loss_rec=0.071
pre_G:[002613/008000], loss_rec=0.071
pre_G:[002614/008000], loss_rec=0.072
pre_G:[002615/008000], loss_rec=0.076
pre_G:[002616/008000], loss_rec=0.066
pre_G:[002617/008000], loss_rec=0.069
pre_G:[002618/008000], loss_rec=0.072
pre_G:[00261

KeyboardInterrupt: 